In [20]:
import ss13_usb as ss13

from typing import NamedTuple
from enum import Enum, IntEnum
from typing_extensions import Self
import importlib
from dataclasses import dataclass

import supermattercore.consts as smc
import supermattercore.graph as smg
import supermattercore.smgraph as smsmg
importlib.reload(smc)
importlib.reload(smg)
importlib.reload(smsmg)
from supermattercore.graph import Direction


from datetime import datetime

In [1]:
ROOT_ROOT = "E:\\Code\\opensource\\games\\"
SCAV_ROOT = ROOT_ROOT+ "ScavStation\\"
NEB_ROOT = ROOT_ROOT + "Nebula\\"
# SCAV_ROOT = "./instance/"

In [17]:
ssm = ss13.Ss13Map(SCAV_ROOT + "maps/ministation/ministation-0.dmm")
with open (SCAV_ROOT + "maps/ministation/ministation-0.dmm", 'rb') as f:
    cur_rev = f.read()
nodes,edges,objs,pipes = smsmg.do_all(ssm)

In [22]:
ssm = ss13.Ss13Map(SCAV_ROOT + "maps/exodus/exodus-2.dmm")
nodes,edges,objs,pipes = smsmg.do_all(ssm, False)

In [18]:
smsmg.plot_graph('Mollusc : <dev>', nodes, edges, 1)

# Git trickery

In [25]:
import git

repo = git.Repo(SCAV_ROOT)

In [35]:
commit = repo.commit("HEAD")
commit_repr = f'[{commit.hexsha[:8]}] @ {datetime.fromtimestamp(commit.authored_date).isoformat(" ")} <{commit.author.name}> "{commit.message.strip()}"'
graph_hist = [("<HEAD:dev> "+commit_repr, nodes, edges)]
smsmg.plot_graph(*graph_hist[0], 1)
print(commit_repr,'\n\t', '-- starting from here')
for commit in repo.iter_commits(all=True, max_count=60, paths='maps/ministation/ministation-0.dmm'):
    commit_repr = f'[{commit.hexsha[:8]}] @ {datetime.fromtimestamp(commit.authored_date).isoformat(" ")} <{commit.author.name}> "{commit.message.strip()}"'
    blobs = [(t.name, t['ministation-0.dmm']) for t in repo.tree(commit)['maps'].trees if 'ministat' in t.name]
    for dirname, blob in blobs:
        ds = blob.data_stream.read()
        if ds != cur_rev:
            fn = f'./instance/maptree/{commit.hexsha[:8]}.dmm'
            cur_rev = ds
            with open(fn, 'wb') as f:
                f.write(ds)
            ssm = ss13.Ss13Map(fn)
            try:
                node2, edge2,_,_ = smsmg.do_all(ssm)
                if nodes!=node2 and edges!=edge2:
                    graph_hist.append((commit_repr, node2, edge2))
                    histlen = len(graph_hist)
                    nodes,edges = node2,edge2
                    smsmg.plot_graph(*graph_hist[histlen-1], histlen)
                    print(commit_repr, '\n\t', '! Nodes/edges CHANGED\n\t\tGraph #', histlen, 'drawn')
                else: 
                    print(commit_repr, ' -- ', 'mapfile changed, nodes/edges equal')
            except AssertionError:
                print(commit_repr, '\n\t', '!!! No SM Core found (???)')
        else: print(commit_repr, ' -- ', 'mapfiles equal')

[2f432d2b] @ 2024-05-23 18:30:25 <Knighted Illithid> "Merge remote-tracking branch 'mynebula/staging' into ci-dev" 
	 -- starting from here
[00660e8a] @ 2024-05-09 03:31:15 <MistakeNot4892> "Fixed the pool on Ministation."  --  mapfiles equal
[d2e8a272] @ 2024-05-08 03:26:37 <MistakeNot4892> "Post-update compile fixes."  --  mapfile changed, nodes/edges equal
[8b1dc629] @ 2024-05-08 03:20:06 <MistakeNot4892> "Merge branch 'main' of github.com:ScavStation/ScavStation into scavdev"  --  mapfile changed, nodes/edges equal
[09c864f2] @ 2024-05-03 22:49:18 <Armyguy> "Yinglet Cam Time"  --  mapfile changed, nodes/edges equal
[5dd26fd6] @ 2024-04-22 10:57:35 <Armyguy> "Fixs the D A R K N E S S"  --  mapfile changed, nodes/edges equal
[c50e97e6] @ 2024-04-20 10:56:43 <Armyguy> "Woo?" 
	 ! Nodes/edges CHANGED
		Graph # 2 drawn
[164dc202] @ 2024-04-20 10:38:13 <Armyguy> "Think thats all of em?"  --  mapfile changed, nodes/edges equal
[1c684ec3] @ 2024-04-20 07:32:18 <Armyguy> "-0 fixed?"  --  ma

# Single impl

In [141]:
core_room = [k for k in ssm.find(smc.VACUUM_FLOOR) if smc.AIRLOCK not in dict(ssm[k])]
smcore = [smg.StoredObj(k, smc.SM_CORE, dict(ssm[k])[smc.SM_CORE], 'SM') for k in core_room if smc.SM_CORE in dict(ssm[k])]
assert len(smcore)>0, "SM core not located"
core_volume = 2.5 * len(core_room)
print(f'{core_volume=} m³')

core_volume=37.5 m³


In [142]:
first = smg.find_amid(ssm, core_room)

stored_objs, pipes0 = smg.split_to_obj_pipes(smg.follow_networks(ssm, first, smcore))

pipenetset = smg.merge_pipes(pipes0)

smsmg.custom_rename(ssm, stored_objs, pipenetset)

In [143]:
pipenetset

PipeNetsSet(segment_map=<381 coordinates>,
	nets_resolve=<44 segments>,
	nets_defs={'black': (385.0 L, 5 tiles, 3 segms) : {'///black': 4, 'manifold//black': 1},
 'black_2': (350.0 L, 4 tiles, 2 segms) : {'///black': 2, 'manifold//black': 2},
 'black_3': (70 L, 1 tiles, 1 segms) : {'///black': 1},
 'cooled': (175.0 L, 2 tiles, 2 segms) : {'///green': 1, 'manifold//green': 1},
 'cooling': (11620.0 L, 164 tiles, 14 segms) : {   '///black': 15,
    '///green': 6,
    '//heat_exchanging': 119,
    '//heat_exchanging/junction': 2,
    '//hidden/black': 4,
    '//hidden/green': 14,
    'manifold//black': 2,
    'manifold//green': 1,
    'manifold/hidden/green': 1},
 'heat_exchanging': (12215.0 L, 173 tiles, 8 segms) : {   '///black': 6,
    '//heat_exchanging': 157,
    '//heat_exchanging/junction': 2,
    '//hidden/black': 5,
    'manifold//black': 3},
 'scalding': (315.0 L, 4 tiles, 2 segms) : {'///yellow': 3, 'manifold//yellow': 1},
 'warm': (2345.0 L, 30 tiles, 12 segms) : {'///cyan': 21

In [144]:
newnodes,newedges=smsmg.gen_graph(stored_objs, pipenetset, True)

SM1 @ (126, 69); d=2; dir_in=<Direction.NORTH: 1>
	2 -- NOCONN
PI1 @ (128, 70); d=4; dir_in=<Direction.WEST: 8>
	4 -- scalding(315.0L)
PO1 @ (124, 70); d=1; dir_in=<Direction.SOUTH: 2>
	1 -- warm(2345.0L)
M1 @ (130, 70); d=2; dir_in=<Direction.NORTH: 1>
	None -- scalding(315.0L)
M2 @ (132, 72); d=2; dir_in=<Direction.NORTH: 1>
	None -- warm(2345.0L)
M3 @ (119, 69); d=2; dir_in=<Direction.NORTH: 1>
	None -- black_2(350.0L)
V1 @ (130, 67); d=1; dir_in=<Direction.SOUTH: 2>
	1 -- scalding(315.0L)
	2 <- cooling(11620.0L)
V2 @ (131, 65); d=4; dir_in=<Direction.WEST: 8>
	8 <- cooling(11620.0L)
	4 -- cooled(175.0L)
V3 @ (139, 76); d=4; dir_in=<Direction.WEST: 8>
	8 <- warm(2345.0L)
	4 -- cooling(11620.0L)
F1 @ (121, 72); d=2; dir_in=<Direction.EAST: 4>
	8 -- black(385.0L)
	1 -- warm(2345.0L)
	2 -- NOCONN
	4 <- warm(2345.0L)
F2 @ (121, 74); d=2; dir_in=<Direction.EAST: 4>
	8 -- black(385.0L)
	1 -- OBJs: ['P1']
	2 -- warm(2345.0L)
	4 <- warm(2345.0L)
P1 @ (121, 75); d=8; dir_in=<Direction.EAST: 